<a href="https://colab.research.google.com/github/lilaiguernelala/Programmation-GPU/blob/main/CorrigeTD1ProgGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2krm4v3w
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2krm4v3w
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-iosnbexc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmp_a71w10v".


# Exercise 3

In [7]:
%%cuda
#include <iostream>
#include <stdio.h>

__global__ void linear_regression(float *inputs, float *targets, int size, float *weights, float *bias, float learning_rate, int epochs) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float cost;
    for (int epoch = 0; epoch < epochs; ++epoch) {
        float predicted = inputs[idx] * weights[0] + bias[0];
        float loss = predicted - targets[idx];
        float d_weights = inputs[idx] * loss / (2 * size);
        float d_bias = loss / (2 * size);
        weights[0] -= learning_rate * d_weights;
        bias[0] -= learning_rate * d_bias;
        cost = sqrtf(powf(loss, 2) / (2 * size));
        if (idx == 0) {
            printf("Iteration: %d | Cost/Loss: %f | Weight: %f | Bias: %f\n", epoch + 1, cost, weights[0], bias[0]);
        }
    }
}

int main(void) {
    int m = 300; // Number of training data points
    int n = 50; // Number of testing data points
    float learning_rate = 0.0001;
    int epochs = 200000;
    float a = 0.5; // y = ax + b
    float b = 2.0;
    int size = m;
    int dataSize = size * sizeof(float);
    float *h_inputs, *h_targets, *h_weights, *h_bias;
    float *d_inputs, *d_targets, *d_weights, *d_bias;

    // Allocate host memory
    h_inputs = (float*)malloc(dataSize);
    h_targets = (float*)malloc(dataSize);
    h_weights = (float*)malloc(sizeof(float));
    h_bias = (float*)malloc(sizeof(float));

    // Generate data
    for (int i = 0; i < m; ++i) {
        h_inputs[i] = i + 1;
        h_targets[i] = (i + 1) * a + b;
    }

    // Allocate device memory
    cudaMalloc((void**)&d_inputs, dataSize);
    cudaMalloc((void**)&d_targets, dataSize);
    cudaMalloc((void**)&d_weights, sizeof(float));
    cudaMalloc((void**)&d_bias, sizeof(float));

    // Copy data to device memory
    cudaMemcpy(d_inputs, h_inputs, dataSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_targets, h_targets, dataSize, cudaMemcpyHostToDevice);

    // Initialize weights and bias
    h_weights[0] = 0;
    h_bias[0] = 0;

    // Copy initial weights and bias to device memory
    cudaMemcpy(d_weights, h_weights, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bias, h_bias, sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    linear_regression<<<1, m>>>(d_inputs, d_targets, size, d_weights, d_bias, learning_rate, epochs);

    // Copy back weights and bias
    cudaMemcpy(h_weights, d_weights, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_bias, d_bias, sizeof(float), cudaMemcpyDeviceToHost);

    printf("Final Weights: %f\n", h_weights[0]);
    printf("Final Bias: %f\n", h_bias[0]);

    // Free device memory
    cudaFree(d_inputs);
    cudaFree(d_targets);
    cudaFree(d_weights);
    cudaFree(d_bias);

    // Free host memory
    free(h_inputs);
    free(h_targets);
    free(h_weights);
    free(h_bias);

    return 0;
}


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Iteration: 45150 | Cost/Loss: 0.077659 | Weight: 0.528270 | Bias: 0.069489
Iteration: 45151 | Cost/Loss: 0.077659 | Weight: 0.528270 | Bias: 0.069489
Iteration: 45152 | Cost/Loss: 0.077659 | Weight: 0.528271 | Bias: 0.069490
Iteration: 45153 | Cost/Loss: 0.077659 | Weight: 0.528271 | Bias: 0.069490
Iteration: 45154 | Cost/Loss: 0.077659 | Weight: 0.528271 | Bias: 0.069490
Iteration: 45155 | Cost/Loss: 0.077659 | Weight: 0.528272 | Bias: 0.069491
Iteration: 45156 | Cost/Loss: 0.077659 | Weight: 0.528272 | Bias: 0.069491
Iteration: 45157 | Cost/Loss: 0.077659 | Weight: 0.528272 | Bias: 0.069491
Iteration: 45158 | Cost/Loss: 0.077658 | Weight: 0.528273 | Bias: 0.069492
Iteration: 45159 | Cost/Loss: 0.077658 | Weight: 0.528273 | Bias: 0.069492
Iteration: 45160 | Cost/Loss: 0.077658 | Weight: 0.528273 | Bias: 0.069492
Iteration: 45161 | Cost/Loss: 0.077658 | Weight: 0.528273 | Bias: 0.069493
Iteration: 45162 | Cos

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Iteration: 180325 | Cost/Loss: 0.074304 | Weight: 0.568555 | Bias: 0.111383
Iteration: 180326 | Cost/Loss: 0.074304 | Weight: 0.568556 | Bias: 0.111383
Iteration: 180327 | Cost/Loss: 0.074304 | Weight: 0.568556 | Bias: 0.111384
Iteration: 180328 | Cost/Loss: 0.074304 | Weight: 0.568556 | Bias: 0.111384
Iteration: 180329 | Cost/Loss: 0.074304 | Weight: 0.568557 | Bias: 0.111384
Iteration: 180330 | Cost/Loss: 0.074304 | Weight: 0.568557 | Bias: 0.111385
Iteration: 180331 | Cost/Loss: 0.074304 | Weight: 0.568557 | Bias: 0.111385
Iteration: 180332 | Cost/Loss: 0.074304 | Weight: 0.568558 | Bias: 0.111385
Iteration: 180333 | Cost/Loss: 0.074304 | Weight: 0.568558 | Bias: 0.111385
Iteration: 180334 | Cost/Loss: 0.074304 | Weight: 0.568558 | Bias: 0.111386
Iteration: 180335 | Cost/Loss: 0.074303 | Weight: 0.568558 | Bias: 0.111386
Iteration: 180336 | Cost/Loss: 0.074303 | Weight: 0.568559 | Bias: 0.111386
Iteration:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__ void linear_regression(float *inputs, float *targets, int size, float *weights, float *bias, float learning_rate, int epochs) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float cost;
    for (int epoch = 0; epoch < epochs; ++epoch) {
        float predicted = inputs[idx] * weights[0] + bias[0];
        float loss = predicted - targets[idx];
        float d_weights = inputs[idx] * loss / (2 * size);
        float d_bias = loss / (2 * size);
        weights[0] -= learning_rate * d_weights;
        bias[0] -= learning_rate * d_bias;
        cost = sqrtf(powf(loss, 2) / (2 * size));
        if (idx == 0) {
            printf("Iteration: %d | Cost/Loss: %f | Weight: %f | Bias: %f\n", epoch + 1, cost, weights[0], bias[0]);
        }
    }
}

int main(void) {
    int m = 300; // Number of training data points
    int n = 50; // Number of testing data points
    float learning_rate = 0.0001;
    int epochs = 200000;
    float a = 0.5; // y = ax + b
    float b = 2.0;
    int size = m;
    int dataSize = size * sizeof(float);
    float *h_inputs, *h_targets, *h_weights, *h_bias;
    float *d_inputs, *d_targets, *d_weights, *d_bias;

    // Allocate host memory
    h_inputs = (float*)malloc(dataSize);
    h_targets = (float*)malloc(dataSize);
    h_weights = (float*)malloc(sizeof(float));
    h_bias = (float*)malloc(sizeof(float));

    // Generate data
    for (int i = 0; i < m; ++i) {
        h_inputs[i] = i + 1;
        h_targets[i] = (i + 1) * a + b;
    }

    // Allocate device memory
    cudaMalloc((void**)&d_inputs, dataSize);
    cudaMalloc((void**)&d_targets, dataSize);
    cudaMalloc((void**)&d_weights, sizeof(float));
    cudaMalloc((void**)&d_bias, sizeof(float));

    // Copy data to device memory
    cudaMemcpy(d_inputs, h_inputs, dataSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_targets, h_targets, dataSize, cudaMemcpyHostToDevice);

    // Initialize weights and bias
    h_weights[0] = 0;
    h_bias[0] = 0;

    // Copy initial weights and bias to device memory
    cudaMemcpy(d_weights, h_weights, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_bias, h_bias, sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    linear_regression<<<1, m>>>(d_inputs, d_targets, size, d_weights, d_bias, learning_rate, epochs);

    // Copy back weights and bias
    cudaMemcpy(h_weights, d_weights, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_bias, d_bias, sizeof(float), cudaMemcpyDeviceToHost);

    printf("Final Weights: %f\n", h_weights[0]);
    printf("Final Bias: %f\n", h_bias[0]);

    // Free device memory
    cudaFree(d_inputs);
    cudaFree(d_targets);
    cudaFree(d_weights);
    cudaFree(d_bias);

    // Free host memory
    free(h_inputs);
    free(h_targets);
    free(h_weights);
    free(h_bias);

    return 0;
}


SyntaxError: invalid syntax (<ipython-input-5-a504d8acd278>, line 5)

# Exercise 4

In [ ]:
%%cuda
#include <iostream>
#include <chrono>

#define N 512
#define Niter 1000

void addCPU(int *a, int *b, int *c) {
    for (int i = 0; i < N*N ; i++) {
        c[i] = a[i] + b[i];
    }
}

__global__ void addKernel1D (int *a, int *b, int *c) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    while (tid < N*N) {
        c[tid] = a[tid] + b[tid];
        tid += blockDim.x * gridDim.x;
    }
}

__global__ void addKernel2D (int *a, int *b, int *c) {
    int tidx = threadIdx.x + blockIdx.x * blockDim.x;
    int tidy = threadIdx.y + blockIdx.y * blockDim.y;
    int id = tidx + tidy * N;
    while (tidx < N && tidy < N) {
        c[id] = a[id] + b[id];
        tidx += blockDim.x * gridDim.x;
        tidy += blockDim.y * gridDim.y;
        id = tidx + tidy * N;
    }
}

int main(void) {
    int A[N*N], B[N*N], C[N*N], D[N*N];
    int *d_A, *d_B, *d_C;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaMalloc(&d_A, N*N*sizeof(int));
    cudaMalloc(&d_B, N*N*sizeof(int));
    cudaMalloc(&d_C, N*N*sizeof(int));

    for (int i = 0 ; i < N*N ; i++) {
            A[i] = i;
            B[i] = i+1;
    }
    cudaMemcpy(d_A, A, N*N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N*N*sizeof(int), cudaMemcpyHostToDevice);

    auto t_start = std::chrono::high_resolution_clock::now();
    for (int i = 0; i <  Niter ; i++) // We do Niter to have an averaged execution time
      addCPU(A, B, C);
    auto t_end = std::chrono::high_resolution_clock::now();

    cudaDeviceProp properties;
    cudaGetDeviceProperties(&properties, 0);
    int nThreads = properties.maxThreadsPerBlock; // Get maximum number of threads per block
    int nBlocks = (N*N + nThreads - 1) / nThreads; // Get right number of blocks to cover our problem

    dim3 gridDim(8,nBlocks/8+1);
    dim3 blockDim(2,nThreads/2);

    addKernel1D<<<nBlocks,nThreads>>>(d_A, d_B, d_C); // WARM-UP

    cudaEventRecord(start);
    for (int i = 0; i <  Niter ; i++) // We do Niter to have an averaged execution time
      addKernel1D<<<nBlocks,nThreads>>>(d_A, d_B, d_C);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    cudaMemcpy(D, d_C, N*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(B, d_B, N*N*sizeof(int), cudaMemcpyDeviceToHost);

    int diff = 0;
    for (int i = 0 ; i < N ; i++) {
        if(D[i] != C[i]) diff = D[i] - C[i];
    }
    if(diff != 0) {
        printf("Wrong computation : diff = %d", diff);
        return 0;
    }
    printf("CPU execution time = %f ms\n", std::chrono::duration<double, std::milli>(t_end-t_start).count()/Niter);
    printf("GPU execution time 1D = %f ms\n", ms/Niter);
    addKernel2D<<<gridDim,blockDim>>>(d_A, d_B, d_C); // WARM-UP

    cudaEventRecord(start);
    for (int i = 0; i <  Niter ; i++) // We do Niter to have an averaged execution time
      addKernel2D<<<gridDim,blockDim>>>(d_A, d_B, d_C);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);

    cudaMemcpy(D, d_C, N*N*sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(B, d_B, N*N*sizeof(int), cudaMemcpyDeviceToHost);

    diff = 0;
    for (int i = 0 ; i < N ; i++) {
        if(D[i] != C[i]) diff = D[i] - C[i];
    }
    if(diff != 0) {
        printf("Wrong computation : diff = %d", diff);
        return 0;
    }
    printf("GPU execution time 2D = %f ms\n", ms/Niter);

  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

}

CPU execution time = 0.754999 ms
GPU execution time 1D = 0.011890 ms
GPU execution time 2D = 0.007938 ms

